<a href="https://colab.research.google.com/github/kavyajeetbora/nlp_rag/blob/master/end_to_end/02_local_llm_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

`colab-xterm` allows you to open a terminal in a cell.

In [1]:
!pip install -q -U transformers python-dotenv langchain langchain_community langchain_huggingface accelerate bitsandbytes unstructured pdfminer randomname chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 140.4 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 13.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 27.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.6/336.6 kB 25.8 

In [41]:
import os
from glob import glob
import shutil
import randomname
from dotenv import load_dotenv
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFacePipeline, HuggingFaceEmbeddings, ChatHuggingFace
from langchain_community.vectorstores import Chroma
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain.retrievers import MultiQueryRetriever
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

Load env variables:

In [3]:
if os.path.exists(".env"):
    os.remove(".env")

from google.colab import files
uploaded = files.upload()
if uploaded:
    if load_dotenv(".env"):
        print("Uploaded and Loaded Sucessfully")

from huggingface_hub import login
login(token=os.environ['HF_TOKEN'])

Saving .env to .env
Uploaded and Loaded Sucessfully


Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


## Downloading the LLM model from HuggingFace

In [4]:
model_id = "microsoft/phi-2"
tokenizer = AutoTokenizer.from_pretrained(model_id)

config = BitsAndBytesConfig(
    load_in_4bit = True
)

model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path = model_id,
    quantization_config = config,
    low_cpu_mem_usage=True
    #attn_implementation="flash_attention_2", # if you have an ampere GPU
)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=100, top_k=50, temperature=0.1)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of the model checkpoint at microsoft/phi-2 were not used when initializing PhiForCausalLM: {'lm_head.bias'}
- This IS expected if you are initializing PhiForCausalLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing PhiForCausalLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Device set to use cuda:0


In [7]:
model_size = model.get_memory_footprint()/1e9
print(f"Model size of {model_id} : {model_size:.2f} GB")

Model size of microsoft/phi-2 : 1.78 GB


In [8]:
num_params = model.num_parameters()/1e9
print(f"Number of parameters: {num_params:.2} Billon")

Number of parameters: 2.8 Billon


# Running Phi-4 model locally

first create the langchain llm pipeline:

In [44]:
hf_pipeline = HuggingFacePipeline(pipeline = pipe)
llm = ChatHuggingFace(llm=llm)

In [46]:
query = "Explain the concept of quantum entanglement in simple terms."
expected_output =  "Quantum entanglement is a phenomenon where two particles become interconnected, so the state of one instantly influences the state of the other, no matter how far apart they are."

llm.invoke(query)

ValueError: Cannot use chat template functions because tokenizer.chat_template is not set and no template argument was passed! For information about writing templates and setting the tokenizer.chat_template attribute, please see the documentation at https://huggingface.co/docs/transformers/main/en/chat_templating

## Let's build a local RAG System

### Ingest the data



In [11]:
!wget -q https://www.niti.gov.in/sites/default/files/2023-02/Annual-Report-2022-2023-English_1.pdf -O annual_report.pdf

In [12]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import OnlinePDFLoader

In [13]:
doc_path = "annual_report.pdf"

if os.path.exists(doc_path):
    loader = PyPDFLoader(doc_path)
    data = loader.load()
    print("PDF is loaded")
else:
    print("PDF file not found")

PDF is loaded


In [14]:
data[10]

Document(metadata={'source': 'annual_report.pdf', 'page': 10}, page_content='MoH&FW Ministry of Health and Family Welfare\nMoHUA Ministry of Housing and Urban Affairs\nMoSPI Ministry of Statistics and Programme Implementation\nNAS National Achievement Survey\nNDC Nationally Determined Contributions\nNEP National Education Policy\nNER North-Eastern Region\nNFHS National Family Health Survey\nNILERD National Institute of Labour Economics Research and Development\nNITI National Institution for Transforming India\nNMP National Monetisation Pipeline\nNOTP National Organ Transplant Programme\nOOMF Output–Outcome Monitoring Framework\nOoSC Out-of-School Children\nPLI Production Linked Incentive\nPMJAY Pradhan Mantri Jan Arogya Yojana\nPSHICMI Policy and Strategy for Health Insurance Coverage of India\nSATH-E Sustainable Action for Transforming Human Capital-Education\nSC-NEC Sub-Committee of National Executive Committee\nSDG Sustainable Development Goals\nSECI State Energy & Climate Index\nSE

### Chunking the information

In [24]:
## Split the text into chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100
)

chunks = text_splitter.split_documents(data)
print(f"The documents has been splitted into {len(chunks)} chunks")

The documents has been splitted into 820 chunks


In [25]:
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={'device': 'cuda:0'},
)

create the vector database

In [26]:
def create_vector_database(chunks: list,name="vector-db"):

    os.makedirs("db", exist_ok=True)

    random_suffix = randomname.get_name()

    persistent_directory = f"db/chroma-{name}-({random_suffix})"

    ## If already there, delete and create a new one

    for folder in glob(f"db/chroma-{name}*"):
        if os.path.exists(folder):
            shutil.rmtree(folder)

    os.mkdir(persistent_directory)

    vector_db = Chroma.from_documents(
        documents = chunks,
        collection_name = name,
        embedding = embeddings,
        persist_directory = persistent_directory
    )

    return vector_db

In [27]:
vector_db = create_vector_database(chunks=chunks, name="annual-report")

## Retrieval

In [28]:
prompt_template = '''You are an AI language model assistant.
Generate 1-5 sub-questions or alternate versions of the given user question to
retrieve relevant documents from a vector database.
Break down questions with multiple concepts into distinct sub-questions.
Provide these alternative questions separated by newlines between XML tags.

Original question: {question}
'''

prompt = PromptTemplate(template = prompt_template, input_variables=["question"])

In [29]:
# retriever = MultiQueryRetriever.from_llm(
#     retriever = vector_db.as_retriever(),
#     llm = llm,
#     prompt = prompt
# )

# retriever = similarity_search_with_score

In [30]:
chat_prompt_template = '''
You are a helpful and friendly assistant.
Answer the user's questions based solely on the context of the provided.
{context}
If the information is not available in the context, respond with "I don't know."

User: {question}
Assistant:
'''

chat_prompt = ChatPromptTemplate.from_template(chat_prompt_template)

## Define the RAG chain

In [40]:
llm

HuggingFacePipeline(pipeline=<transformers.pipelines.text_generation.TextGenerationPipeline object at 0x7e28be7332d0>, model_id='microsoft/phi-2')

In [39]:
retriever = vector_db.as_retriever()

query = 'What is the full form of RSNA ?'
retriever.invoke(query)

[Document(metadata={'page': 118, 'source': 'annual_report.pdf'}, page_content='Department of Consumer Affairs, Department of Food and Public Distribution System, Department of \nPersonnel and Training, Department of Pensions and Pensioners’ Welfare, Department of Administrative \nReforms and Public Grievances, Department of Legal Affairs and Department of Justice. The Research \nDivision oversees the Research Scheme of NITI Aayog (or RSNA), which aims at supporting various \nresearch studies.\nCoverage and Beneficiary under National Food Security Act (NFSA), 2013'),
 Document(metadata={'page': 120, 'source': 'annual_report.pdf'}, page_content='New Research Scheme of NITI Aayog (RSNA) Guidelines 2021\nIn line with NITI’s mandate to position itself as a knowledge and innovation hub, new set of guidelines, \nviz. ‘Research Scheme of NITI Aayog 2021’ has been launched. The revamped guidelines aim at broad-\nbasing research work including Institutional & individual based research, besides a

In [36]:
rag_chain = (
    {"context": vector_db.as_retriever(), "question": RunnablePassthrough()}
    | chat_prompt
    | llm
    | StrOutputParser()
)

In [37]:
%%time

query = 'What is this document about ?'

response = rag_chain.invoke(query)

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


CPU times: user 5.33 s, sys: 238 ms, total: 5.56 s
Wall time: 5.56 s


In [35]:
response

'Human: \nYou are a helpful and friendly assistant.\nAnswer the user\'s questions based solely on the context of the provided.\n[Document(metadata={\'page\': 69, \'source\': \'annual_report.pdf\'}, page_content=\'Chef Anahita Dhondy conducting a live millet cooking demonstration session during 4th \\nIn-house NITI lecture\\nFRONTIER TECHNOLOGIES\\nResponsible Use of Artificial Intelligence (AI)\\nIn 2018, NITI Aayog released the National Strategy for Artificial Intelligence (NSAI). Based on the \\nrecommendations of NSAI, two papers were subsequently released following inter-ministerial and \\nvarious stakeholder consultations in the domain of Responsible AI — Part 1: Principles of Responsible\'), Document(metadata={\'page\': 98, \'source\': \'annual_report.pdf\'}, page_content=\'robust ecosystem in each of India’s scheduled languages.\\nIndia being a multilingual nation, it is only apt to provide an opportunity to every innovator to innovate \\nand ideate in one’s own language of expr